# Examen d'ocvx 2

## Section 1

### question 1.

In [46]:
import warnings
import math
import seaborn as sns
import numpy as np
from matplotlib import pyplot as plt

In [40]:
class test_function():
    
    def __init__(self, name, value, dim, nb_params, convex=None, grad=None, constraints = None):
        self.name = name
        self.value = value
        self.dim = dim
        self.nb_params = nb_params
        self.convex = convex
        self.grad = grad
        self.constraints = constraints

In [41]:
mini_1_d = {
    "name": "mini_1",
    "dim" : 1,
    "nb_params" : 1,
    "convex" : True,
    "value" : (lambda x: (x**2) - x + 2),
    "grad"  : (lambda x: 2*x - 1),
    "constraints": []
}

mini_1 = test_function(**mini_1_d)

In [42]:
maxi_1_d = {
    "name": "mini_1",
    "dim" : 1,
    "nb_params" : 1,
    "convex" : True,
    "value" : (lambda x: -1*(x**2) + x +1),
    "grad"  : (lambda x: -2*x + 1),
    "constraints": []
}

maxi_1 = test_function(**maxi_1_d)

In [43]:
#C'est la fonction de Bashar, faut changer un peu

func_3_d = {
    "name": "function_3",
    "dim" : 2,
    "nb_params" : 1,
    "convex" : True ,
    "value" : (lambda x, gamma: x[:, 0]**2 + gamma*x[:, 1]**2),
    "grad"  : (lambda x, gamma: np.array([2*x[:, 0], 2*gamma*x[:, 1]]).reshape(1, -1)),
    "constraints" : [],
}

func_3 = test_function(**func_3_d)

In [48]:
exp_1_d = {
    "name": "exp_1",
    "dim" : 1,
    "nb_params" : 1,
    "convex" : True,
    "value" : (lambda x: math.exp(x)+math.exp(-x)-1),
    "grad"  : (lambda x: math.exp(x)-math.exp(-x)),
    "constraints": []
}

exp_1 = test_function(**exp_1_d)

### question 2

In [49]:
def newton(x, function, epsilon=0.000001):
    
    diff = 1
    
    while diff > epsilon:
        
        xn = x - function["value"](x)/function["grad"](x)
        diff = abs(xn - x)
        x = xn
        print(diff)
    
    print(xn)

In [ ]:
def Newton_meth(orig_point, function, step):
    next_point = function["value"](orig_point)
    




orig = 2 #





